In [1]:
import numpy as np
import pandas as pd
import xlrd
import os
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV,\
cross_val_score, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier,\
ExtraTreesClassifier, VotingClassifier, StackingRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder,\
OrdinalEncoder, MaxAbsScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, f1_score,\
accuracy_score, classification_report, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from xgboost import XGBClassifier
import xgboost as xgb
from matplotlib import pyplot as plt
import warnings

In [2]:
# Importing CSV of NCAAW regular season team data, pulled from Kaggle
season_df = pd.read_csv('/Users/aheinke/Documents/Flatiron/NYC-DS-010923/Phase_3/Phase3_Proj/PROJ_CSVs/WRegularSeasonDetailedResults.csv')
season_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,11,3103,63,3237,49,H,0,23,54,...,13,6,10,11,27,11,23,7,6,19
1,2010,11,3104,73,3399,68,N,0,26,62,...,21,14,27,14,26,7,20,4,2,27
2,2010,11,3110,71,3224,59,A,0,29,62,...,14,19,23,17,23,8,15,6,0,15
3,2010,11,3111,63,3267,58,A,0,27,52,...,26,16,25,22,22,15,11,14,5,14
4,2010,11,3119,74,3447,70,H,1,30,74,...,17,11,21,21,32,12,14,4,2,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70002,2023,113,3268,96,3234,68,H,0,34,71,...,38,11,15,18,19,14,24,5,3,14
70003,2023,113,3385,69,3163,64,A,0,23,63,...,21,12,15,11,25,14,12,5,7,19
70004,2023,113,3416,57,3396,53,H,0,19,60,...,26,14,17,16,19,5,9,10,9,20
70005,2023,113,3437,67,3177,64,H,0,24,67,...,19,8,11,21,26,13,12,9,2,14


In [3]:
tourney_df = pd.read_csv('/Users/aheinke/Documents/Flatiron/NYC-DS-010923/Phase_3/Phase3_Proj/PROJ_CSVs/WNCAATourneyDetailedResults.csv')
tourney_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,138,3124,69,3201,55,N,0,28,57,...,34,3,5,17,19,12,18,4,1,18
1,2010,138,3173,67,3395,66,N,0,23,59,...,27,14,15,18,26,8,8,8,6,22
2,2010,138,3181,72,3214,37,H,0,26,57,...,15,3,8,10,21,4,16,6,4,20
3,2010,138,3199,75,3256,61,H,0,25,63,...,20,17,22,16,21,13,16,5,4,24
4,2010,138,3207,62,3265,42,N,0,24,68,...,26,11,17,16,22,9,10,3,4,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2022,147,3163,91,3301,87,N,2,37,77,...,23,16,19,6,30,20,13,4,7,16
756,2022,147,3257,62,3276,50,N,0,25,58,...,14,15,20,10,24,9,21,6,2,12
757,2022,151,3163,63,3390,58,N,0,21,57,...,23,8,13,11,23,10,11,11,3,16
758,2022,151,3376,72,3257,59,N,0,27,57,...,8,4,7,11,18,5,15,13,2,17


In [4]:
# Merging the two datasets, sorting by the season and the day number of the game so that
# the games line up in order
merged_df = pd.concat([season_df, tourney_df])

# sort the merged dataframe by "col_name" in ascending order
sorted_df = merged_df.sort_values(['Season', 'DayNum'], ascending = [True, True])

# reset the index
games_df = sorted_df.reset_index(drop = True)
games_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,11,3103,63,3237,49,H,0,23,54,...,13,6,10,11,27,11,23,7,6,19
1,2010,11,3104,73,3399,68,N,0,26,62,...,21,14,27,14,26,7,20,4,2,27
2,2010,11,3110,71,3224,59,A,0,29,62,...,14,19,23,17,23,8,15,6,0,15
3,2010,11,3111,63,3267,58,A,0,27,52,...,26,16,25,22,22,15,11,14,5,14
4,2010,11,3119,74,3447,70,H,1,30,74,...,17,11,21,21,32,12,14,4,2,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,2023,113,3268,96,3234,68,H,0,34,71,...,38,11,15,18,19,14,24,5,3,14
70763,2023,113,3385,69,3163,64,A,0,23,63,...,21,12,15,11,25,14,12,5,7,19
70764,2023,113,3416,57,3396,53,H,0,19,60,...,26,14,17,16,19,5,9,10,9,20
70765,2023,113,3437,67,3177,64,H,0,24,67,...,19,8,11,21,26,13,12,9,2,14


In [5]:
# Creating a new column for the difference in points for each game
games_df['ScoreGap'] = games_df['WScore'] - games_df['LScore']
games_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,ScoreGap
0,2010,11,3103,63,3237,49,H,0,23,54,...,6,10,11,27,11,23,7,6,19,14
1,2010,11,3104,73,3399,68,N,0,26,62,...,14,27,14,26,7,20,4,2,27,5
2,2010,11,3110,71,3224,59,A,0,29,62,...,19,23,17,23,8,15,6,0,15,12
3,2010,11,3111,63,3267,58,A,0,27,52,...,16,25,22,22,15,11,14,5,14,5
4,2010,11,3119,74,3447,70,H,1,30,74,...,11,21,21,32,12,14,4,2,14,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,2023,113,3268,96,3234,68,H,0,34,71,...,11,15,18,19,14,24,5,3,14,28
70763,2023,113,3385,69,3163,64,A,0,23,63,...,12,15,11,25,14,12,5,7,19,5
70764,2023,113,3416,57,3396,53,H,0,19,60,...,14,17,16,19,5,9,10,9,20,4
70765,2023,113,3437,67,3177,64,H,0,24,67,...,8,11,21,26,13,12,9,2,14,3


In [6]:
seeds_df = pd.read_csv('/Users/aheinke/Documents/Flatiron/NYC-DS-010923/Phase_3/Phase3_Proj/PROJ_CSVs/WNCAATourneySeeds.csv')
seeds_df

,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163
2,1998,W03,3112
3,1998,W04,3301
4,1998,W05,3272
...,...,...,...
1535,2022,Z12,3125
1536,2022,Z13,3138
1537,2022,Z14,3110
1538,2022,Z15,3218


In [7]:
seeds_df['SeedNum'] = seeds_df['Seed'].str.extract('(\d+)').astype(int)
seeds_df

,Season,Seed,TeamID,SeedNum
0,1998,W01,3330,1
1,1998,W02,3163,2
2,1998,W03,3112,3
3,1998,W04,3301,4
4,1998,W05,3272,5
...,...,...,...,...
1535,2022,Z12,3125,12
1536,2022,Z13,3138,13
1537,2022,Z14,3110,14
1538,2022,Z15,3218,15


In [8]:
team_season_seed_dict = dict(zip(zip(seeds_df['TeamID'], seeds_df['Season']), seeds_df['SeedNum']))
team_season_seed_dict

{(3330, 1998): 1,
 (3163, 1998): 2,
 (3112, 1998): 3,
 (3301, 1998): 4,
 (3272, 1998): 5,
 (3438, 1998): 6,
 (3208, 1998): 7,
 (3307, 1998): 8,
 (3304, 1998): 9,
 (3203, 1998): 10,
 (3374, 1998): 11,
 (3464, 1998): 12,
 (3263, 1998): 13,
 (3365, 1998): 14,
 (3193, 1998): 15,
 (3384, 1998): 16,
 (3403, 1998): 1,
 (3104, 1998): 2,
 (3256, 1998): 3,
 (3345, 1998): 4,
 (3179, 1998): 5,
 (3155, 1998): 6,
 (3417, 1998): 7,
 (3283, 1998): 8,
 (3323, 1998): 9,
 (3276, 1998): 10,
 (3274, 1998): 11,
 (3161, 1998): 12,
 (3449, 1998): 13,
 (3221, 1998): 14,
 (3422, 1998): 15,
 (3212, 1998): 16,
 (3397, 1998): 1,
 (3314, 1998): 2,
 (3228, 1998): 3,
 (3235, 1998): 4,
 (3353, 1998): 5,
 (3435, 1998): 6,
 (3198, 1998): 7,
 (3443, 1998): 8,
 (3372, 1998): 9,
 (3266, 1998): 10,
 (3364, 1998): 11,
 (3332, 1998): 12,
 (3245, 1998): 13,
 (3453, 1998): 14,
 (3224, 1998): 15,
 (3251, 1998): 16,
 (3390, 1998): 1,
 (3181, 1998): 2,
 (3196, 1998): 3,
 (3234, 1998): 4,
 (3242, 1998): 5,
 (3458, 1998): 6,
 (3428,

In [9]:
games_df['WSeed'] = games_df.apply(lambda row: team_season_seed_dict.get((row['WTeamID'], row['Season']),\
                                                                           0), axis = 1)

In [10]:
games_df['LSeed'] = games_df.apply(lambda row: team_season_seed_dict.get((row['LTeamID'], row['Season']),\
                                                                           0), axis = 1)

In [11]:
games_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,ScoreGap,WSeed,LSeed
0,2010,11,3103,63,3237,49,H,0,23,54,...,11,27,11,23,7,6,19,14,0,0
1,2010,11,3104,73,3399,68,N,0,26,62,...,14,26,7,20,4,2,27,5,0,0
2,2010,11,3110,71,3224,59,A,0,29,62,...,17,23,8,15,6,0,15,12,0,0
3,2010,11,3111,63,3267,58,A,0,27,52,...,22,22,15,11,14,5,14,5,0,0
4,2010,11,3119,74,3447,70,H,1,30,74,...,21,32,12,14,4,2,14,4,0,0


In [12]:
total_season_wins = games_df.groupby(['Season', 'WTeamID']).count()
total_season_wins = total_season_wins.reset_index()[['Season', 'WTeamID', 'DayNum']].rename(columns = {'DayNum':\
                                                                                    'NumWins', 'WTeamID': 'TeamID'})
total_season_wins.head()

,Season,TeamID,NumWins
0,2010,3102,1
1,2010,3103,17
2,2010,3104,11
3,2010,3105,14
4,2010,3106,12


In [13]:
total_season_losses = games_df.groupby(['Season', 'LTeamID']).count()
total_season_losses = total_season_losses.reset_index()[['Season', 'LTeamID', 'DayNum']].rename(columns = {'DayNum':\
                                                                                    'NumLosses', 'LTeamID': 'TeamID'})
total_season_losses.head()

,Season,TeamID,NumLosses
0,2010,3102,27
1,2010,3103,13
2,2010,3104,18
3,2010,3105,13
4,2010,3106,17


In [14]:
gap_win = games_df.groupby(['Season', 'WTeamID']).mean().reset_index()
gap_win = gap_win[['Season', 'WTeamID', 'ScoreGap']].rename(columns = {'ScoreGap': 'GapWins', 'WTeamID': 'TeamID'})
gap_win.head()

,Season,TeamID,GapWins
0,2010,3102,20.000000
1,2010,3103,13.176471
2,2010,3104,15.363636
3,2010,3105,7.357143
4,2010,3106,8.500000


In [15]:
gap_loss = games_df.groupby(['Season', 'LTeamID']).mean().reset_index()
gap_loss = gap_loss[['Season', 'LTeamID', 'ScoreGap']].rename(columns = {'ScoreGap': 'GapLosses', 'LTeamID': 'TeamID'})
gap_loss.head()

,Season,TeamID,GapLosses
0,2010,3102,21.444444
1,2010,3103,11.230769
2,2010,3104,14.000000
3,2010,3105,16.153846
4,2010,3106,9.882353


In [16]:
df_features_season_w = games_df.groupby(['Season', 'WTeamID']).count().reset_index()[['Season',\
                                                'WTeamID']].rename(columns = {'WTeamID': 'TeamID'})
df_features_season_l = games_df.groupby(['Season', 'LTeamID']).count().reset_index()[['Season',\
                                                'LTeamID']].rename(columns = {'LTeamID': 'TeamID'})

In [17]:
df_features_season = pd.concat([df_features_season_w, df_features_season_l],\
                               0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop = True)

In [18]:
df_features_season = df_features_season.merge(total_season_wins, on = ['Season', 'TeamID'], how = 'left')
df_features_season = df_features_season.merge(total_season_losses, on = ['Season', 'TeamID'], how = 'left')
df_features_season = df_features_season.merge(gap_win, on = ['Season', 'TeamID'], how = 'left')
df_features_season = df_features_season.merge(gap_loss, on = ['Season', 'TeamID'], how = 'left')

In [19]:
df_features_season.fillna(0, inplace = True)  

In [20]:
df_features_season['WinRatio'] = 100 * (df_features_season['NumWins'] / (df_features_season['NumWins'] + \
                                                                         df_features_season['NumLosses']))
df_features_season['GapAvg'] = (
    (df_features_season['NumWins'] * df_features_season['GapWins'] - 
    df_features_season['NumLosses'] * df_features_season['GapLosses'])
    / (df_features_season['NumWins'] + df_features_season['NumLosses'])
)

In [21]:
df_features_season

,Season,TeamID,NumWins,NumLosses,GapWins,GapLosses,WinRatio,GapAvg
0,2010,3102,1.0,27.0,20.000000,21.444444,3.571429,-19.964286
1,2010,3103,17.0,13.0,13.176471,11.230769,56.666667,2.600000
2,2010,3104,11.0,18.0,15.363636,14.000000,37.931034,-2.862069
3,2010,3105,14.0,13.0,7.357143,16.153846,51.851852,-3.962963
4,2010,3106,12.0,17.0,8.500000,9.882353,41.379310,-2.275862
...,...,...,...,...,...,...,...,...
4875,2023,3473,1.0,22.0,9.000000,19.318182,4.347826,-18.086957
4876,2023,3474,5.0,18.0,10.200000,19.888889,21.739130,-13.347826
4877,2023,3475,8.0,16.0,13.875000,11.625000,33.333333,-3.125000
4878,2023,3476,7.0,19.0,10.571429,13.368421,26.923077,-6.923077


In [22]:
games_features_df = pd.merge(
    games_df,
    df_features_season,
    how = 'left',
    left_on = ['Season', 'WTeamID'],
    right_on = ['Season', 'TeamID']
).rename(columns = {
    'NumWins': 'NumWinsW',
    'NumLosses': 'NumLossesW',
    'GapWins': 'GapWinsW',
    'GapLosses': 'GapLossesW',
    'WinRatio': 'WinRatioW',
    'GapAvg': 'GapAvgW',
}).drop(columns = 'TeamID', axis = 1)

In [23]:
games_features_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LPF,ScoreGap,WSeed,LSeed,NumWinsW,NumLossesW,GapWinsW,GapLossesW,WinRatioW,GapAvgW
0,2010,11,3103,63,3237,49,H,0,23,54,...,19,14,0,0,17.0,13.0,13.176471,11.230769,56.666667,2.600000
1,2010,11,3104,73,3399,68,N,0,26,62,...,27,5,0,0,11.0,18.0,15.363636,14.000000,37.931034,-2.862069
2,2010,11,3110,71,3224,59,A,0,29,62,...,15,12,0,0,13.0,8.0,16.230769,10.500000,61.904762,6.047619
3,2010,11,3111,63,3267,58,A,0,27,52,...,14,5,0,0,19.0,12.0,11.263158,11.166667,61.290323,2.580645
4,2010,11,3119,74,3447,70,H,1,30,74,...,14,4,0,0,8.0,16.0,9.500000,14.437500,33.333333,-6.458333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,2023,113,3268,96,3234,68,H,0,34,71,...,14,28,0,0,23.0,5.0,16.478261,15.600000,82.142857,10.750000
70763,2023,113,3385,69,3163,64,A,0,23,63,...,19,5,0,0,20.0,7.0,15.050000,17.000000,74.074074,6.740741
70764,2023,113,3416,57,3396,53,H,0,19,60,...,20,4,0,0,12.0,13.0,14.916667,18.384615,48.000000,-2.400000
70765,2023,113,3437,67,3177,64,H,0,24,67,...,14,3,0,0,24.0,5.0,18.041667,10.400000,82.758621,13.137931


In [24]:
games_features_df = pd.merge(
    games_features_df,
    df_features_season,
    how = 'left',
    left_on = ['Season', 'LTeamID'],
    right_on = ['Season', 'TeamID']
).rename(columns = {
    'NumWins': 'NumWinsL',
    'NumLosses': 'NumLossesL',
    'GapWins': 'GapWinsL',
    'GapLosses': 'GapLossesL',
    'WinRatio': 'WinRatioL',
    'GapAvg': 'GapAvgL',
}).drop(columns = 'TeamID', axis = 1)

In [25]:
games_features_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,GapWinsW,GapLossesW,WinRatioW,GapAvgW,NumWinsL,NumLossesL,GapWinsL,GapLossesL,WinRatioL,GapAvgL
0,2010,11,3103,63,3237,49,H,0,23,54,...,13.176471,11.230769,56.666667,2.600000,2.0,26.0,15.000000,21.230769,7.142857,-18.642857
1,2010,11,3104,73,3399,68,N,0,26,62,...,15.363636,14.000000,37.931034,-2.862069,13.0,16.0,14.692308,16.312500,44.827586,-2.413793
2,2010,11,3110,71,3224,59,A,0,29,62,...,16.230769,10.500000,61.904762,6.047619,15.0,14.0,7.200000,13.214286,51.724138,-2.655172
3,2010,11,3111,63,3267,58,A,0,27,52,...,11.263158,11.166667,61.290323,2.580645,13.0,16.0,7.846154,11.375000,44.827586,-2.758621
4,2010,11,3119,74,3447,70,H,1,30,74,...,9.500000,14.437500,33.333333,-6.458333,9.0,21.0,8.333333,13.190476,30.000000,-6.733333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,2023,113,3268,96,3234,68,H,0,34,71,...,16.478261,15.600000,82.142857,10.750000,22.0,6.0,24.090909,10.333333,78.571429,16.714286
70763,2023,113,3385,69,3163,64,A,0,23,63,...,15.050000,17.000000,74.074074,6.740741,24.0,5.0,22.375000,7.400000,82.758621,17.241379
70764,2023,113,3416,57,3396,53,H,0,19,60,...,14.916667,18.384615,48.000000,-2.400000,10.0,16.0,11.100000,11.312500,38.461538,-2.692308
70765,2023,113,3437,67,3177,64,H,0,24,67,...,18.041667,10.400000,82.758621,13.137931,15.0,14.0,21.000000,11.642857,51.724138,5.241379


TSP IMAGE

In [26]:
# true shooting percentage
games_features_df['1SP'] = 100 * ((0.5 * games_features_df['WScore']) / (games_features_df['WFGA'] + (0.475 * \
                                                                                games_features_df['WFTA'])))
games_features_df['2SP'] = 100 * ((0.5 * games_features_df['LScore']) / (games_features_df['LFGA'] + (0.475 * \
                                                                                games_features_df['LFTA'])))

In [27]:
games_features_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,WinRatioW,GapAvgW,NumWinsL,NumLossesL,GapWinsL,GapLossesL,WinRatioL,GapAvgL,1SP,2SP
0,2010,11,3103,63,3237,49,H,0,23,54,...,56.666667,2.600000,2.0,26.0,15.000000,21.230769,7.142857,-18.642857,49.980167,41.702128
1,2010,11,3104,73,3399,68,N,0,26,62,...,37.931034,-2.862069,13.0,16.0,14.692308,16.312500,44.827586,-2.413793,48.472776,44.840092
2,2010,11,3110,71,3224,59,A,0,29,62,...,61.904762,6.047619,15.0,14.0,7.200000,13.214286,51.724138,-2.655172,52.437223,42.800145
3,2010,11,3111,63,3267,58,A,0,27,52,...,61.290323,2.580645,13.0,16.0,7.846154,11.375000,44.827586,-2.758621,55.975122,33.770015
4,2010,11,3119,74,3447,70,H,1,30,74,...,33.333333,-6.458333,9.0,21.0,8.333333,13.190476,30.000000,-6.733333,46.702430,41.679071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,2023,113,3268,96,3234,68,H,0,34,71,...,82.142857,10.750000,22.0,6.0,24.090909,10.333333,78.571429,16.714286,61.440000,46.495726
70763,2023,113,3385,69,3163,64,A,0,23,63,...,74.074074,6.740741,24.0,5.0,22.375000,7.400000,82.758621,17.241379,48.218029,46.292948
70764,2023,113,3416,57,3396,53,H,0,19,60,...,48.000000,-2.400000,10.0,16.0,11.100000,11.312500,38.461538,-2.692308,39.134912,33.941723
70765,2023,113,3437,67,3177,64,H,0,24,67,...,82.758621,13.137931,15.0,14.0,21.000000,11.642857,51.724138,5.241379,45.193929,38.917604


In [28]:
# Grouping by Team and Season to obtain the mean for our necessary stats
group = games_features_df.groupby(['Season','WTeamID'])
games_features = group.mean()
games_features

DayNum     WScore      LTeamID     LScore     NumOT  \
Season WTeamID                                                           
2010   3102     32.000000  68.000000  3152.000000  48.000000  0.000000   
       3103     74.941176  66.352941  3247.647059  53.176471  0.058824   
       3104     57.181818  72.818182  3267.000000  57.454545  0.000000   
       3105     87.000000  66.428571  3217.500000  59.071429  0.000000   
       3106     88.666667  59.833333  3230.666667  51.333333  0.083333   
...                   ...        ...          ...        ...       ...   
2023   3473     96.000000  84.000000  3404.000000  75.000000  0.000000   
       3474     62.200000  68.600000  3334.400000  58.400000  0.000000   
       3475     63.000000  71.750000  3359.625000  57.875000  0.000000   
       3476     64.714286  65.571429  3227.285714  55.000000  0.000000   
       3477     79.090909  73.454545  3271.090909  62.454545  0.000000   

                     WFGM       WFGA      WFGM3      WFGA3       WFTM  ...  \
Season WTeamID                                                         ...   
2010   3102     23.000000  51.000000   6.000000  14.000000  16.000000  ...   
       3103     23.941176  56.000000   4.352941  12.235294  14.117647  ...   
       3104     28.000000  59.545455   4.818182  13.636364  12.000000  ...   
       3105     21.785714  47.571429   3.142857   8.357143  19.714286  ...   
       3106     19.583333  49.833333   2.250000   7.000000  18.416667  ...   
...                   ...        ...        ...        ...        ...  ...   
2023   3473     29.000000  47.000000  14.000000  26.000000  12.000000  ...   
       3474     23.800000  59.000000   6.800000  20.200000  14.200000  ...   
       3475     24.875000  59.375000   5.375000  16.625000  16.625000  ...   
       3476     24.285714  56.142857   7.000000  19.000000  10.000000  ...   
       3477     26.818182  59.090909   4.272727  14.545455  15.545455  ...   

                WinRatioW    GapAvgW   NumWinsL  NumLossesL   GapWinsL  \
Season WTeamID                                                           
2010   3102      3.571429 -19.964286   9.000000   18.000000  14.000000   
       3103     56.666667   2.600000  10.000000   19.176471  11.981285   
       3104     37.931034  -2.862069  11.000000   17.727273  12.561448   
       3105     51.851852  -3.962963  10.500000   17.500000  10.469473   
       3106     41.379310  -2.275862   9.750000   18.583333  10.527159   
...                   ...        ...        ...         ...        ...   
2023   3473      4.347826 -18.086957  11.000000   16.000000  14.181818   
       3474     21.739130 -13.347826   7.000000   17.800000   8.768571   
       3475     33.333333  -3.125000   6.500000   18.250000  10.018750   
       3476     26.923077  -6.923077   5.571429   19.285714   9.977381   
       3477     42.307692  -5.000000   7.363636   16.272727  10.243934   

                GapLossesL  WinRatioL    GapAvgL        1SP        2SP  
Season WTeamID                                                          
2010   3102      14.500000  33.333333  -5.000000  55.760558  38.277512  
       3103      14.173460  34.613575  -5.626469  51.040934  41.037520  
       3104      15.092754  38.134376  -4.569409  53.028467  42.252070  
       3105      15.154323  37.237395  -5.490715  55.354892  43.509581  
       3106      15.717463  34.290277  -6.754353  47.706953  40.726426  
...                    ...        ...        ...        ...        ...  
2023   3473      11.625000  40.740741  -1.111111  76.923077  47.483381  
       3474      19.879650  28.281159 -12.364498  50.234025  48.307636  
       3475      18.460807  26.100845 -11.348105  51.485078  44.592368  
       3476      17.542286  22.366300 -11.481300  51.840605  43.478205  
       3477      17.159796  31.456381  -8.477458  53.165958  48.006340  

[4860 rows x 48 columns]

In [29]:
games_features_df2 = games_features.reset_index()

In [30]:
games_features_df2 = games_features_df2[['Season', 'WTeamID', 'WScore', 'LScore', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl',\
                                         'WBlk', 'WPF', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'ScoreGap']]

In [31]:
games_features_df2

,Season,WTeamID,WScore,LScore,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,ScoreGap
0,2010,3102,68.000000,48.000000,11.000000,29.000000,13.000000,14.000000,9.000000,1.000000,10.000000,14.000000,19.000000,13.000000,18.000000,7.000000,5.000000,17.000000,20.000000
1,2010,3103,66.352941,53.176471,14.176471,25.764706,14.235294,16.882353,8.705882,2.705882,16.058824,14.823529,20.764706,9.764706,19.058824,8.882353,3.705882,18.176471,13.176471
2,2010,3104,72.818182,57.454545,14.363636,28.000000,17.181818,20.545455,8.454545,3.000000,18.454545,14.818182,21.454545,10.818182,22.181818,8.636364,2.818182,18.181818,15.363636
3,2010,3105,66.428571,59.071429,11.285714,25.571429,11.285714,25.714286,8.928571,5.642857,18.714286,15.357143,18.428571,12.571429,22.142857,12.071429,3.285714,22.428571,7.357143
4,2010,3106,59.833333,51.333333,14.916667,25.583333,12.166667,19.250000,7.500000,5.333333,21.750000,14.166667,19.750000,7.416667,17.833333,8.000000,3.416667,22.583333,8.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4855,2023,3473,84.000000,75.000000,5.000000,25.000000,26.000000,25.000000,8.000000,6.000000,15.000000,13.000000,12.000000,12.000000,13.000000,17.000000,1.000000,18.000000,9.000000
4856,2023,3474,68.600000,58.400000,10.000000,22.400000,9.600000,13.000000,8.600000,2.000000,19.600000,7.200000,23.000000,8.200000,17.600000,5.800000,2.800000,17.600000,10.200000
4857,2023,3475,71.750000,57.875000,11.000000,27.000000,14.750000,18.000000,8.375000,3.750000,18.500000,7.875000,21.250000,9.250000,19.000000,8.625000,3.375000,21.125000,13.875000
4858,2023,3476,65.571429,55.000000,8.428571,24.857143,15.571429,14.000000,4.857143,2.714286,14.714286,7.714286,20.571429,7.571429,13.285714,6.000000,3.000000,16.714286,10.571429


In [32]:
# merging all
merged_df = pd.merge(games_features_df, games_features_df2, how = 'left', on = ['Season', 'WTeamID', 'WScore', \
                                'LScore', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF',\
                    'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'ScoreGap'])
merged_df

/Users/aheinke/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,WinRatioW,GapAvgW,NumWinsL,NumLossesL,GapWinsL,GapLossesL,WinRatioL,GapAvgL,1SP,2SP
0,2010,11,3103,63,3237,49,H,0,23,54,...,56.666667,2.600000,2.0,26.0,15.000000,21.230769,7.142857,-18.642857,49.980167,41.702128
1,2010,11,3104,73,3399,68,N,0,26,62,...,37.931034,-2.862069,13.0,16.0,14.692308,16.312500,44.827586,-2.413793,48.472776,44.840092
2,2010,11,3110,71,3224,59,A,0,29,62,...,61.904762,6.047619,15.0,14.0,7.200000,13.214286,51.724138,-2.655172,52.437223,42.800145
3,2010,11,3111,63,3267,58,A,0,27,52,...,61.290323,2.580645,13.0,16.0,7.846154,11.375000,44.827586,-2.758621,55.975122,33.770015
4,2010,11,3119,74,3447,70,H,1,30,74,...,33.333333,-6.458333,9.0,21.0,8.333333,13.190476,30.000000,-6.733333,46.702430,41.679071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,2023,113,3268,96,3234,68,H,0,34,71,...,82.142857,10.750000,22.0,6.0,24.090909,10.333333,78.571429,16.714286,61.440000,46.495726
70763,2023,113,3385,69,3163,64,A,0,23,63,...,74.074074,6.740741,24.0,5.0,22.375000,7.400000,82.758621,17.241379,48.218029,46.292948
70764,2023,113,3416,57,3396,53,H,0,19,60,...,48.000000,-2.400000,10.0,16.0,11.100000,11.312500,38.461538,-2.692308,39.134912,33.941723
70765,2023,113,3437,67,3177,64,H,0,24,67,...,82.758621,13.137931,15.0,14.0,21.000000,11.642857,51.724138,5.241379,45.193929,38.917604


In [33]:
# Creating a GameID column for reference once we slpit our rows below
merged_df['GameID'] = merged_df.index
merged_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,GapAvgW,NumWinsL,NumLossesL,GapWinsL,GapLossesL,WinRatioL,GapAvgL,1SP,2SP,GameID
0,2010,11,3103,63,3237,49,H,0,23,54,...,2.600000,2.0,26.0,15.000000,21.230769,7.142857,-18.642857,49.980167,41.702128,0
1,2010,11,3104,73,3399,68,N,0,26,62,...,-2.862069,13.0,16.0,14.692308,16.312500,44.827586,-2.413793,48.472776,44.840092,1
2,2010,11,3110,71,3224,59,A,0,29,62,...,6.047619,15.0,14.0,7.200000,13.214286,51.724138,-2.655172,52.437223,42.800145,2
3,2010,11,3111,63,3267,58,A,0,27,52,...,2.580645,13.0,16.0,7.846154,11.375000,44.827586,-2.758621,55.975122,33.770015,3
4,2010,11,3119,74,3447,70,H,1,30,74,...,-6.458333,9.0,21.0,8.333333,13.190476,30.000000,-6.733333,46.702430,41.679071,4


In [34]:
# Creating a winners dataframe with all of the necessary columns which will be concatenated with
# a similar losers column to create our final dataframe
# (excluding the 'WFGM' column, as this is already accounted for by the 'WScore')
cols_to_keep = ['GameID', 'Season', 'DayNum', 'WTeamID', 'WSeed', 'WScore', 'WLoc', 'NumOT', 'WFGA', 'WFGM3',\
                'WFGA3', 'WFTM', 'WFTA', '1SP', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'ScoreGap',\
                'NumWinsW', 'NumLossesW', 'GapWinsW', 'GapLossesW', 'WinRatioW', 'GapAvgW']
winners_df = merged_df[cols_to_keep]
winners_df

,GameID,Season,DayNum,WTeamID,WSeed,WScore,WLoc,NumOT,WFGA,WFGM3,...,WStl,WBlk,WPF,ScoreGap,NumWinsW,NumLossesW,GapWinsW,GapLossesW,WinRatioW,GapAvgW
0,0,2010,11,3103,0,63,H,0,54,5,...,7,0,15,14,17.0,13.0,13.176471,11.230769,56.666667,2.600000
1,1,2010,11,3104,0,73,N,0,62,5,...,5,2,25,5,11.0,18.0,15.363636,14.000000,37.931034,-2.862069
2,2,2010,11,3110,0,71,A,0,62,6,...,6,2,17,12,13.0,8.0,16.230769,10.500000,61.904762,6.047619
3,3,2010,11,3111,0,63,A,0,52,4,...,5,10,18,5,19.0,12.0,11.263158,11.166667,61.290323,2.580645
4,4,2010,11,3119,0,74,H,1,74,7,...,5,3,18,4,8.0,16.0,9.500000,14.437500,33.333333,-6.458333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,70762,2023,113,3268,0,96,H,0,71,14,...,15,1,14,28,23.0,5.0,16.478261,15.600000,82.142857,10.750000
70763,70763,2023,113,3385,0,69,A,0,63,9,...,4,6,18,5,20.0,7.0,15.050000,17.000000,74.074074,6.740741
70764,70764,2023,113,3416,0,57,H,0,60,4,...,3,7,14,4,12.0,13.0,14.916667,18.384615,48.000000,-2.400000
70765,70765,2023,113,3437,0,67,H,0,67,8,...,7,9,10,3,24.0,5.0,18.041667,10.400000,82.758621,13.137931


In [35]:
# Creating our Win/Lose column, where a win is represented by a '1'
winners_df['Win1_Lose0'] = 1

<ipython-input-35-5baecc7e1ad9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winners_df['Win1_Lose0'] = 1


In [36]:
# Renaming each of the columns so that we can easily append the two new dataframes
dict = {'WTeamID': 'TeamID', 'WSeed': 'Seed', 'WScore': 'Score', 'WFGA': 'FGA', 'WFGM3': 'FGM3',\
                'WFGA3': 'FGA3', 'WFTM': 'FTM', 'WFTA': 'FTA', '1SP': 'SP', 'WOR': 'OR', 'WDR': 'DR', 'WAst': 'Ast',\
        'WTO': 'TO', 'WStl': 'Stl', 'WBlk': 'Blk', 'WPF': 'PF', 'NumWinsW': 'NumWins', 'NumLossesW': 'NumLosses',\
        'GapWinsW': 'GapWins', 'GapLossesW': 'GapLosses', 'WinRatioW': 'WinRatio', 'GapAvgW': 'GapAvg'}

In [37]:
winners_df.rename(columns = dict, inplace = True)
winners_df

/Users/aheinke/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,GameID,Season,DayNum,TeamID,Seed,Score,WLoc,NumOT,FGA,FGM3,...,Blk,PF,ScoreGap,NumWins,NumLosses,GapWins,GapLosses,WinRatio,GapAvg,Win1_Lose0
0,0,2010,11,3103,0,63,H,0,54,5,...,0,15,14,17.0,13.0,13.176471,11.230769,56.666667,2.600000,1
1,1,2010,11,3104,0,73,N,0,62,5,...,2,25,5,11.0,18.0,15.363636,14.000000,37.931034,-2.862069,1
2,2,2010,11,3110,0,71,A,0,62,6,...,2,17,12,13.0,8.0,16.230769,10.500000,61.904762,6.047619,1
3,3,2010,11,3111,0,63,A,0,52,4,...,10,18,5,19.0,12.0,11.263158,11.166667,61.290323,2.580645,1
4,4,2010,11,3119,0,74,H,1,74,7,...,3,18,4,8.0,16.0,9.500000,14.437500,33.333333,-6.458333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,70762,2023,113,3268,0,96,H,0,71,14,...,1,14,28,23.0,5.0,16.478261,15.600000,82.142857,10.750000,1
70763,70763,2023,113,3385,0,69,A,0,63,9,...,6,18,5,20.0,7.0,15.050000,17.000000,74.074074,6.740741,1
70764,70764,2023,113,3416,0,57,H,0,60,4,...,7,14,4,12.0,13.0,14.916667,18.384615,48.000000,-2.400000,1
70765,70765,2023,113,3437,0,67,H,0,67,8,...,9,10,3,24.0,5.0,18.041667,10.400000,82.758621,13.137931,1


In [38]:
# Creating the losers dataframe with all of the necessary columns 
# (excluding the 'LFGM' column, as this is already accounted for by the 'LScore')
cols_to_keep = ['GameID', 'Season', 'DayNum', 'LTeamID', 'LSeed', 'LScore', 'WLoc', 'NumOT', 'LFGA', 'LFGM3',\
                'LFGA3', 'LFTM', 'LFTA', '2SP', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'ScoreGap',\
                'NumWinsL', 'NumLossesL', 'GapWinsL', 'GapLossesL', 'WinRatioL', 'GapAvgL']
losers_df = merged_df[cols_to_keep]
losers_df

,GameID,Season,DayNum,LTeamID,LSeed,LScore,WLoc,NumOT,LFGA,LFGM3,...,LStl,LBlk,LPF,ScoreGap,NumWinsL,NumLossesL,GapWinsL,GapLossesL,WinRatioL,GapAvgL
0,0,2010,11,3237,0,49,H,0,54,3,...,7,6,19,14,2.0,26.0,15.000000,21.230769,7.142857,-18.642857
1,1,2010,11,3399,0,68,N,0,63,4,...,4,2,27,5,13.0,16.0,14.692308,16.312500,44.827586,-2.413793
2,2,2010,11,3224,0,59,A,0,58,2,...,6,0,15,12,15.0,14.0,7.200000,13.214286,51.724138,-2.655172
3,3,2010,11,3267,0,58,A,0,74,6,...,14,5,14,5,13.0,16.0,7.846154,11.375000,44.827586,-2.758621
4,4,2010,11,3447,0,70,H,1,74,9,...,4,2,14,4,9.0,21.0,8.333333,13.190476,30.000000,-6.733333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,70762,2023,113,3234,0,68,H,0,66,11,...,5,3,14,28,22.0,6.0,24.090909,10.333333,78.571429,16.714286
70763,70763,2023,113,3163,0,64,A,0,62,8,...,5,7,19,5,24.0,5.0,22.375000,7.400000,82.758621,17.241379
70764,70764,2023,113,3396,0,53,H,0,70,7,...,10,9,20,4,10.0,16.0,11.100000,11.312500,38.461538,-2.692308
70765,70765,2023,113,3177,0,64,H,0,77,4,...,9,2,14,3,15.0,14.0,21.000000,11.642857,51.724138,5.241379


In [39]:
# Creating our Win/Lose column, where a loss is represented by a '0'
losers_df['Win1_Lose0'] = 0

<ipython-input-39-794fc49922ac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  losers_df['Win1_Lose0'] = 0


In [40]:
# Renaming each of the columns so that we can easily append the two new dataframes
dict = {'LTeamID': 'TeamID', 'LSeed': 'Seed', 'LScore': 'Score', 'LFGA': 'FGA', 'LFGM3': 'FGM3',\
            'LFGA3': 'FGA3', 'LFTM': 'FTM', 'LFTA': 'FTA', '2SP': 'SP', 'LOR': 'OR', 'LDR': 'DR', 'LAst': 'Ast',\
        'LTO': 'TO', 'LStl': 'Stl', 'LBlk': 'Blk', 'LPF': 'PF', 'NumWinsL': 'NumWins', 'NumLossesL': 'NumLosses',\
        'GapWinsL': 'GapWins', 'GapLossesL': 'GapLosses', 'WinRatioL': 'WinRatio', 'GapAvgL': 'GapAvg'}

In [41]:
losers_df.rename(columns = dict, inplace = True)
losers_df

/Users/aheinke/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,GameID,Season,DayNum,TeamID,Seed,Score,WLoc,NumOT,FGA,FGM3,...,Blk,PF,ScoreGap,NumWins,NumLosses,GapWins,GapLosses,WinRatio,GapAvg,Win1_Lose0
0,0,2010,11,3237,0,49,H,0,54,3,...,6,19,14,2.0,26.0,15.000000,21.230769,7.142857,-18.642857,0
1,1,2010,11,3399,0,68,N,0,63,4,...,2,27,5,13.0,16.0,14.692308,16.312500,44.827586,-2.413793,0
2,2,2010,11,3224,0,59,A,0,58,2,...,0,15,12,15.0,14.0,7.200000,13.214286,51.724138,-2.655172,0
3,3,2010,11,3267,0,58,A,0,74,6,...,5,14,5,13.0,16.0,7.846154,11.375000,44.827586,-2.758621,0
4,4,2010,11,3447,0,70,H,1,74,9,...,2,14,4,9.0,21.0,8.333333,13.190476,30.000000,-6.733333,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,70762,2023,113,3234,0,68,H,0,66,11,...,3,14,28,22.0,6.0,24.090909,10.333333,78.571429,16.714286,0
70763,70763,2023,113,3163,0,64,A,0,62,8,...,7,19,5,24.0,5.0,22.375000,7.400000,82.758621,17.241379,0
70764,70764,2023,113,3396,0,53,H,0,70,7,...,9,20,4,10.0,16.0,11.100000,11.312500,38.461538,-2.692308,0
70765,70765,2023,113,3177,0,64,H,0,77,4,...,2,14,3,15.0,14.0,21.000000,11.642857,51.724138,5.241379,0


In [42]:
# Merging our winning and losing dataframes into a new dataframe that we can run through our models!
merged_df2 = pd.concat([winners_df, losers_df])
ncaaw_df = merged_df2.sort_values(['GameID', 'Season'], ascending = [True, True])
ncaaw_df

,GameID,Season,DayNum,TeamID,Seed,Score,WLoc,NumOT,FGA,FGM3,...,Blk,PF,ScoreGap,NumWins,NumLosses,GapWins,GapLosses,WinRatio,GapAvg,Win1_Lose0
0,0,2010,11,3103,0,63,H,0,54,5,...,0,15,14,17.0,13.0,13.176471,11.230769,56.666667,2.600000,1
0,0,2010,11,3237,0,49,H,0,54,3,...,6,19,14,2.0,26.0,15.000000,21.230769,7.142857,-18.642857,0
1,1,2010,11,3104,0,73,N,0,62,5,...,2,25,5,11.0,18.0,15.363636,14.000000,37.931034,-2.862069,1
1,1,2010,11,3399,0,68,N,0,63,4,...,2,27,5,13.0,16.0,14.692308,16.312500,44.827586,-2.413793,0
2,2,2010,11,3110,0,71,A,0,62,6,...,2,17,12,13.0,8.0,16.230769,10.500000,61.904762,6.047619,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70764,70764,2023,113,3396,0,53,H,0,70,7,...,9,20,4,10.0,16.0,11.100000,11.312500,38.461538,-2.692308,0
70765,70765,2023,113,3437,0,67,H,0,67,8,...,9,10,3,24.0,5.0,18.041667,10.400000,82.758621,13.137931,1
70765,70765,2023,113,3177,0,64,H,0,77,4,...,2,14,3,15.0,14.0,21.000000,11.642857,51.724138,5.241379,0
70766,70766,2023,113,3466,0,75,A,0,62,12,...,3,16,33,9.0,14.0,15.777778,14.714286,39.130435,-2.782609,1


In [43]:
ncaaw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141534 entries, 0 to 70766
Data columns (total 29 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   GameID      141534 non-null  int64  
 1   Season      141534 non-null  int64  
 2   DayNum      141534 non-null  int64  
 3   TeamID      141534 non-null  int64  
 4   Seed        141534 non-null  int64  
 5   Score       141534 non-null  int64  
 6   WLoc        141534 non-null  object 
 7   NumOT       141534 non-null  int64  
 8   FGA         141534 non-null  int64  
 9   FGM3        141534 non-null  int64  
 10  FGA3        141534 non-null  int64  
 11  FTM         141534 non-null  int64  
 12  FTA         141534 non-null  int64  
 13  SP          141534 non-null  float64
 14  OR          141534 non-null  int64  
 15  DR          141534 non-null  int64  
 16  Ast         141534 non-null  int64  
 17  TO          141534 non-null  int64  
 18  Stl         141534 non-null  int64  
 19  Blk

In [44]:
wloc_map = {'H': 1, 'A': 2, 'N': 0}
ncaaw_df['WLoc'] = ncaaw_df['WLoc'].map(wloc_map)
ncaaw_df

,GameID,Season,DayNum,TeamID,Seed,Score,WLoc,NumOT,FGA,FGM3,...,Blk,PF,ScoreGap,NumWins,NumLosses,GapWins,GapLosses,WinRatio,GapAvg,Win1_Lose0
0,0,2010,11,3103,0,63,1,0,54,5,...,0,15,14,17.0,13.0,13.176471,11.230769,56.666667,2.600000,1
0,0,2010,11,3237,0,49,1,0,54,3,...,6,19,14,2.0,26.0,15.000000,21.230769,7.142857,-18.642857,0
1,1,2010,11,3104,0,73,0,0,62,5,...,2,25,5,11.0,18.0,15.363636,14.000000,37.931034,-2.862069,1
1,1,2010,11,3399,0,68,0,0,63,4,...,2,27,5,13.0,16.0,14.692308,16.312500,44.827586,-2.413793,0
2,2,2010,11,3110,0,71,2,0,62,6,...,2,17,12,13.0,8.0,16.230769,10.500000,61.904762,6.047619,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70764,70764,2023,113,3396,0,53,1,0,70,7,...,9,20,4,10.0,16.0,11.100000,11.312500,38.461538,-2.692308,0
70765,70765,2023,113,3437,0,67,1,0,67,8,...,9,10,3,24.0,5.0,18.041667,10.400000,82.758621,13.137931,1
70765,70765,2023,113,3177,0,64,1,0,77,4,...,2,14,3,15.0,14.0,21.000000,11.642857,51.724138,5.241379,0
70766,70766,2023,113,3466,0,75,2,0,62,12,...,3,16,33,9.0,14.0,15.777778,14.714286,39.130435,-2.782609,1


DayNum
103    1613
89     1611
110    1592
96     1592
75     1555
       ... 
99      122
53      109
132      74
2        45
3        33
Length: 132, dtype: int64

In [45]:
final_ncaaw_df = ncaaw_df.loc[ncaaw_df['Season'] < 2023]
final_ncaaw_df

,GameID,Season,DayNum,TeamID,Seed,Score,WLoc,NumOT,FGA,FGM3,...,Blk,PF,ScoreGap,NumWins,NumLosses,GapWins,GapLosses,WinRatio,GapAvg,Win1_Lose0
0,0,2010,11,3103,0,63,1,0,54,5,...,0,15,14,17.0,13.0,13.176471,11.230769,56.666667,2.600000,1
0,0,2010,11,3237,0,49,1,0,54,3,...,6,19,14,2.0,26.0,15.000000,21.230769,7.142857,-18.642857,0
1,1,2010,11,3104,0,73,0,0,62,5,...,2,25,5,11.0,18.0,15.363636,14.000000,37.931034,-2.862069,1
1,1,2010,11,3399,0,68,0,0,63,4,...,2,27,5,13.0,16.0,14.692308,16.312500,44.827586,-2.413793,0
2,2,2010,11,3110,0,71,2,0,62,6,...,2,17,12,13.0,8.0,16.230769,10.500000,61.904762,6.047619,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66166,66166,2022,151,3390,1,58,0,0,66,4,...,3,16,5,32.0,4.0,19.312500,4.250000,88.888889,16.694444,0
66167,66167,2022,151,3376,1,72,0,0,57,6,...,4,11,13,35.0,2.0,21.457143,1.500000,94.594595,20.216216,1
66167,66167,2022,151,3257,1,59,0,0,63,1,...,2,17,13,29.0,5.0,19.862069,5.400000,85.294118,16.147059,0
66168,66168,2022,153,3376,1,64,0,0,60,3,...,4,11,15,35.0,2.0,21.457143,1.500000,94.594595,20.216216,1


In [63]:
X = final_ncaaw_df.drop(['Win1_Lose0'], axis=1)
y = final_ncaaw_df['Win1_Lose0']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size=0.2, random_state=42)

scaler = StandardScaler()

# Fit the scaler on the training data
scaler.fit(X_train)

# Transform the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create an instance of XGBoost classifier
xgb_model = xgb.XGBClassifier()

# Fit the model on the scaled training data
xgb_model.fit(X_train_scaled, y_train)

# Predict on the scaled testing data
y_pred = xgb_model.predict(X_test_scaled)

In [64]:
print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

0.8958742632612967
0.8978584447144593
0.8933806861115309
0.8956139686387395


In [65]:
now_final_ncaaw_df = ncaaw_df[ncaaw_df['Season'] == 2023]
now_final_ncaaw_df

,GameID,Season,DayNum,TeamID,Seed,Score,WLoc,NumOT,FGA,FGM3,...,Blk,PF,ScoreGap,NumWins,NumLosses,GapWins,GapLosses,WinRatio,GapAvg,Win1_Lose0
66169,66169,2023,7,3102,0,67,2,0,61,6,...,4,24,4,10.0,15.0,10.800000,10.533333,40.000000,-2.000000,1
66169,66169,2023,7,3176,0,63,2,0,57,5,...,5,17,4,11.0,16.0,8.545455,15.437500,40.740741,-5.666667,0
66170,66170,2023,7,3104,0,98,1,0,58,13,...,7,23,47,20.0,7.0,20.850000,13.571429,74.074074,11.925926,1
66170,66170,2023,7,3105,0,51,1,0,64,2,...,2,20,47,13.0,12.0,9.384615,18.833333,52.000000,-4.160000,0
66171,66171,2023,7,3113,0,69,1,0,72,4,...,5,17,1,7.0,18.0,9.571429,15.111111,28.000000,-8.200000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70764,70764,2023,113,3396,0,53,1,0,70,7,...,9,20,4,10.0,16.0,11.100000,11.312500,38.461538,-2.692308,0
70765,70765,2023,113,3437,0,67,1,0,67,8,...,9,10,3,24.0,5.0,18.041667,10.400000,82.758621,13.137931,1
70765,70765,2023,113,3177,0,64,1,0,77,4,...,2,14,3,15.0,14.0,21.000000,11.642857,51.724138,5.241379,0
70766,70766,2023,113,3466,0,75,2,0,62,12,...,3,16,33,9.0,14.0,15.777778,14.714286,39.130435,-2.782609,1


In [49]:
now_final_ncaaw_df.columns.values.tolist()

['GameID',
 'Season',
 'DayNum',
 'TeamID',
 'Seed',
 'Score',
 'WLoc',
 'NumOT',
 'FGA',
 'FGM3',
 'FGA3',
 'FTM',
 'FTA',
 'SP',
 'OR',
 'DR',
 'Ast',
 'TO',
 'Stl',
 'Blk',
 'PF',
 'ScoreGap',
 'NumWins',
 'NumLosses',
 'GapWins',
 'GapLosses',
 'WinRatio',
 'GapAvg',
 'Win1_Lose0']

In [66]:
import itertools
# Define the column you want to generate all possible pairs for
column_name = 'TeamID'

# Get all unique values in the column
unique_values = now_final_ncaaw_df[column_name].unique()

# Generate all possible pairs of values
value_pairs = list(itertools.combinations(unique_values, 2))

# Create a DataFrame to store the predictions for each pair
predictions_df = pd.DataFrame(columns = ['Team' + '_1', 'Team' + '_2', 'Prediction'])

# Loop through each pair of values and make a prediction
for pair in value_pairs:
    # Create a new DataFrame containing the features for the matchup you want to predict
    matchup_features = pd.DataFrame({
        'GameID': [0, 1, 2, 3],
        'Season': [0, 0, 0, 0],
        'DayNum': [0, 0, 0, 0],
        'TeamID': [pair[0], pair[1], pair[0], pair[1]],
        'Score': [0, 0, 0, 0],
        'Seed': [0, 0, 0, 0],
        'WLoc': [0, 0, 0, 0],
        'NumOT': [0, 0, 0, 0],
        'FGA': [0, 0, 0, 0],
        'FGM3': [0, 0, 0, 0],
        'FGA3': [0, 0, 0, 0],
        'FTM': [0, 0, 0, 0],
        'FTA': [0, 0, 0, 0],
        'SP': [0, 0, 0, 0],
        'OR': [0, 0, 0, 0],
        'DR': [0, 0, 0, 0],
        'Ast': [0, 0, 0, 0],
        'TO': [0, 0, 0, 0],
        'Stl': [0, 0, 0, 0],
        'Blk': [0, 0, 0, 0],
        'PF': [0, 0, 0, 0],
        'ScoreGap': [0, 0, 0, 0],
        'NumWins': [0, 0, 0, 0],
        'NumLosses': [0, 0, 0, 0],
        'GapWins': [0, 0, 0, 0],
        'GapLosses': [0, 0, 0, 0],
        'WinRatio': [0, 0, 0, 0],
        'GapAvg': [0, 0, 0, 0],
    })

    # Replace prediction = xgb_model.predict(matchup_features) with the following line
    prediction = xgb_model.predict_proba(matchup_features)

    # Extract the probability of the positive class (i.e., team 1 wins)
    proba_team_1_wins = prediction[0, 1]

    # Append the prediction to the DataFrame
    predictions_df = predictions_df.append({
        'Team' + '_1': pair[0],
        'Team' + '_2': pair[1],
        'Prediction': proba_team_1_wins
    }, ignore_index = True)

# Print the predictions DataFrame
predictions_df

,Team_1,Team_2,Prediction
0,3102.0,3176.0,0.499871
1,3102.0,3104.0,0.499871
2,3102.0,3105.0,0.499871
3,3102.0,3113.0,0.499871
4,3102.0,3319.0,0.499871
...,...,...,...
64975,3194.0,3470.0,0.499871
64976,3194.0,3317.0,0.499871
64977,3412.0,3470.0,0.499871
64978,3412.0,3317.0,0.499871


In [73]:
predictions_df.value_counts('Prediction')

Prediction
0.499871    64980
dtype: int64

In [68]:
X = final_ncaaw_df.drop(['Win1_Lose0'], axis=1)
y = final_ncaaw_df['Win1_Lose0']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.15, random_state = 42)

steps = [('imputer', SimpleImputer(strategy="median")), ('std_scaler', StandardScaler()),
        ('rf_clf', DecisionTreeClassifier(random_state=42))]

pipeline = Pipeline(steps)


# Train the pipeline (tranformations & predictor)
pipeline.fit(X_train, y_train)

# Predict using the pipeline (includes the transfomers & trained predictor)
predicted = pipeline.predict(X_test)
predicted

array([1, 0, 0, ..., 1, 1, 0])

In [69]:
def cross_validation_pip(X_train, y_train, estimator, num_split = 10):
    
    X_train = X_train.values
    y_train = y_train.values
    
    score_train_list = []
    score_val_list = []
    
    for train_index, valid_index in KFold(n_splits = num_split).split(X_train):
        
        # train and validation splitting 
        X_train_fold, X_val_fold = X_train[train_index], X_train[valid_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[valid_index]

        estimator.fit(X_train_fold, y_train_fold)
        
        # now how did we do?
        accuracy_train = estimator.score(X_train_fold, y_train_fold)
        accuracy_val = estimator.score(X_val_fold, y_val_fold)
        score_val_list.append(accuracy_val)
        score_train_list.append(accuracy_train)
    
    return {'train': np.mean(score_train_list), 'validation': np.mean(score_val_list)}

In [70]:
cross_validation_pip(X_train, y_train, pipeline)

{'train': 1.0, 'validation': 0.8047685446229644}